In [1]:
import numpy as np
import kaiwu as kw
kw.license.init(user_id="72317291601100802", sdk_code="vDSsMrcS1XvoHxrKEyWGPu3y6bydtx")
from kaiwu.classical import SimulatedAnnealingOptimizer

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# First Model

In [ ]:
import numpy as np
import random
import math
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a simple CNN model for CIFAR-10
def build_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer for 10 classes
    return model

# Compile the model
model = build_cnn_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (for initial weights)
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Extract the trained weights (kernel and bias) from the first Conv2D layer
conv2d_layer = model.layers[0]  # Assuming the first Conv2D layer
kernel_weights, bias_weights = conv2d_layer.get_weights()  # Extract kernel and bias

# binarized_kernel_weights = np.sign(kernel_weights)  # Binarize the kernel weights to +1 or -1
# bias_weights will be left unchanged in this case
binarized_kernel_weights = kernel_weights


Epoch 1/5
782/782 [==============================] - 8s 10ms/step - loss: 1.4954 - accuracy: 0.4618 - val_loss: 1.3843 - val_accuracy: 0.5197
Epoch 2/5
782/782 [==============================] - 7s 9ms/step - loss: 1.1419 - accuracy: 0.5999 - val_loss: 1.0623 - val_accuracy: 0.6301
Epoch 3/5
782/782 [==============================] - 7s 8ms/step - loss: 0.9931 - accuracy: 0.6543 - val_loss: 0.9822 - val_accuracy: 0.6578
Epoch 4/5
782/782 [==============================] - 7s 8ms/step - loss: 0.9002 - accuracy: 0.6862 - val_loss: 0.9449 - val_accuracy: 0.6705
Epoch 5/5
782/782 [==============================] - 7s 8ms/step - loss: 0.8374 - accuracy: 0.7087 - val_loss: 0.9362 - val_accuracy: 0.6770


In [ ]:
predictions = model.predict(x_test)  # x_test is the CIFAR-10 test data
predicted_labels = np.argmax(predictions, axis=1)  # Get the index with the highest probability for each sample
true_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predicted_labels == true_labels)  # Compare predicted vs. true labels

# Print the accuracy
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 2ms/step
Test Set Accuracy: 67.70%


In [ ]:
def generate_qubo_matrix(weights):
    num_weights = len(weights)
    Q = np.zeros((num_weights, num_weights))
    
    # Penalize large weights and encourage sparsity
    for i in range(num_weights):
        Q[i, i] = abs(weights[i])  # Larger weights are penalized more
    
    return Q

flattened_kernel_weights = binarized_kernel_weights.flatten()
num_variables = min(len(flattened_kernel_weights), 600)  # Limit number of binary variables to 600
selected_weights = flattened_kernel_weights[:num_variables]
Q = generate_qubo_matrix(selected_weights)
Q.shape

(600, 600)

In [ ]:
solver = SimulatedAnnealingOptimizer()

# Solve the QUBO problem
solution = solver.solve(Q)
best_solution = solution[0]

In [ ]:
pruned_weights = np.array([selected_weights[i] if best_solution[i] == 1 else 0 for i in range(len(selected_weights))])

# Update the model with the pruned weights
pruned_kernel_weights = np.copy(binarized_kernel_weights)
pruned_kernel_weights.flatten()[:num_variables] = pruned_weights  # Apply the pruned weights
# pruned_kernel_weights.flatten()[selection] = pruned_weights  # Apply the pruned weights

model.layers[0].set_weights([np.reshape(pruned_kernel_weights, model.layers[0].get_weights()[0].shape), bias_weights])

# Optionally fine-tune the pruned model
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Second Model

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define the deep learning model (a simple CNN)
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define a function to create a more complex QUBO matrix
def create_complex_qubo(weights, sparsity_factor=0.1, interaction_factor=0.01, entropy_factor=0.1):
    # n_weights = len(weights)
    n_weights = 600
    Q = np.zeros((n_weights, n_weights))
    
    # Add sparsity regularization (penalize large weights, encourage sparsity)
    for i in range(n_weights):
        Q[i, i] += sparsity_factor * weights[i] ** 2  # Penalize non-zero weights
    
    # Add interaction regularization (encourage some weights to be correlated or anticorrelated)
    for i in range(n_weights):
        for j in range(i + 1, n_weights):
            Q[i, j] += interaction_factor * weights[i] * weights[j]  # Penalize uncorrelated weights
    
    # Add entropy minimization (encourage confident predictions)
    for i in range(n_weights):
        for j in range(i + 1, n_weights):
            Q[i, j] += entropy_factor * (weights[i] - 0.5) * (weights[j] - 0.5)  # Enforce weights to be either 0 or 1
    
    return Q

# Define the energy function for QUBO
def qubo_energy(x, Q):
    return np.dot(x.T, np.dot(Q, x))  # Calculate the energy (objective function)

# Initialize the model and get the weights
model = create_model()
initial_weights = []
initial_biases = []
for layer in model.layers:
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
        kernel, bias = layer.get_weights()
        initial_weights.append(kernel.flatten())  # Flatten kernels (weights)
        initial_biases.append(bias.flatten())    # Flatten biases

# Concatenate the initial weights and biases
initial_weights = np.concatenate(initial_weights)
initial_biases = np.concatenate(initial_biases)


# Create a more complex QUBO matrix
binary_weights_all = np.zeros(1)
for i in range(int(initial_weights.shape[0] / 600)):
    Q = create_complex_qubo(initial_weights)
    solver = SimulatedAnnealingOptimizer()

    # Solve the QUBO problem
    solution = solver.solve(Q)
    best_solution = solution[0]
    
    optimized_x = best_solution
    binary_weights = np.zeros(600)
    binary_weights[optimized_x == 1] = initial_weights[i*600:i*600+600][optimized_x == 1]
    binary_weights_all = np.concatenate((binary_weights_all, binary_weights))

binary_weights_all_new = binary_weights_all[1:]
binary_weights_all_new = np.concatenate((binary_weights_all_new, np.array([0] * (len(initial_weights) - len(binary_weights_all_new)))))

In [ ]:
# Reconstruct weights and biases for each layer
layer_idx_w = 0
layer_idx_b = 0
updated_weights = []
updated_biases = []
for layer in model.layers:
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
        num_params = layer.get_weights()[0].size  # Kernel size
        num_biases = layer.get_weights()[1].size  # Bias size

        # Get the kernel weights for the layer
        layer_kernel = binary_weights_all_new[layer_idx_w:layer_idx_w + num_params].reshape(layer.get_weights()[0].shape)
        updated_weights.append(layer_kernel)

        # Get the biases for the layer
        # layer_bias = binary_weights_all_new[layer_idx + num_params:layer_idx + num_params + num_biases].reshape(layer.get_weights()[1].shape)
        layer_bias = initial_biases[layer_idx_b:layer_idx_b + num_biases].reshape(layer.get_weights()[1].shape)
        updated_biases.append(layer_bias)

        # Move to the next set of weights
        # layer_idx += num_params + num_biases
        layer_idx_w += num_params
        layer_idx_b += num_biases

# Now set the weights and biases to the model
updated_weights = [w for w in updated_weights]
updated_biases = [b for b in updated_biases]

# Set the new weights and biases for the model
model.set_weights([w for pair in zip(updated_weights, updated_biases) for w in pair])

# Verify the model summary after setting the weights
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 2304)              0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.8721 - accuracy: 0.6985 - val_loss: 0.9224 - val_accuracy: 0.6854
Epoch 2/10
782/782 [==============================] - 7s 8ms/step - loss: 0.8116 - accuracy: 0.7188 - val_loss: 0.9131 - val_accuracy: 0.6830
Epoch 3/10
782/782 [==============================] - 7s 8ms/step - loss: 0.7592 - accuracy: 0.7380 - val_loss: 0.8808 - val_accuracy: 0.6974
Epoch 4/10
782/782 [==============================] - 7s 8ms/step - loss: 0.7123 - accuracy: 0.7540 - val_loss: 0.8825 - val_accuracy: 0.7001
Epoch 5/10
782/782 [==============================] - 7s 8ms/step - loss: 0.6692 - accuracy: 0.7675 - val_loss: 0.8942 - val_accuracy: 0.7003
Epoch 6/10
782/782 [==============================] - 7s 8ms/step - loss: 0.6276 - accuracy: 0.7830 - val_loss: 0.9391 - val_accuracy: 0.6890
Epoch 7/10
782/782 [==============================] - 7s 8ms/step - loss: 0.5932 - accuracy: 0.7958 - val_loss: 0.8945 - val_accuracy: 0.7076
Epoch 